In [1]:
import torch
import numpy as np
from utils import get_freest_gpu
import torch.nn.functional as F
from new_block_env_new import NewBlockEnv
import stable_baselines3
import sys

In [2]:
from predict_transformer_nonlinear import NonLinearNet

In [3]:
freest_gpu = get_freest_gpu()
freest_gpu

1

In [4]:
torch.cuda.set_device(f'cuda:{freest_gpu}')

In [5]:
state_hidden_size = 2048
num_state_layers = 2
trajectory_hidden_size = 2048
num_trajectory_layers = 2
mlp = False
trajectory_sigmoid = False
lstm = False
repeat_trajectory_calculations = False
num_classes = 4

In [6]:
net = NonLinearNet(3, 3, state_hidden_size, 64, trajectory_hidden_size, num_classes, num_trajectory_layers, num_state_layers, mlp=mlp, trajectory_sigmoid=trajectory_sigmoid, lstm=lstm, repeat_trajectory_calculations=repeat_trajectory_calculations).cuda()

In [ ]:
# net = NonLinearNet(3, 3, 64, 64, mlp=True).cuda()

In [7]:
model_name = "more_layers.parameters"

In [8]:
net.load_state_dict(torch.load(model_name))
net.eval()

NonLinearNet(
  (trajectory_encoder): TrajectoryNetTransformer(
    (positional_encoder): PositionalEncoder()
    (encoder_layer): TransformerEncoderLayer(
      (self_attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=100, out_features=100, bias=True)
      )
      (linear1): Linear(in_features=100, out_features=2048, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (linear2): Linear(in_features=2048, out_features=100, bias=True)
      (norm1): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
    )
    (transformer): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=100, out_features=100, bias=True)
          )
        

In [9]:
traj_encoder = net.trajectory_encoder
state_encoder = net.state_encoder

In [10]:
NUM_EXAMPLES = 10100

In [11]:
states = torch.Tensor(np.load(f'states_{NUM_EXAMPLES}.npy')).to(torch.int64)
states = F.one_hot(states, num_classes=4).view(-1, 150, 100).to(torch.float)

In [13]:
ground_truths = np.load(f"weights_{NUM_EXAMPLES}.npy")

In [39]:
ground_truths[0]

array([-0.88798511, -0.12189739,  0.44342246])

In [20]:
for idx, ground_truth in enumerate(ground_truths):
    if all(ground_truth < 0):
        break

In [40]:
print(ground_truth)
rewards[idx]

[-0.57261853 -0.78653656 -0.23123203]


tensor([-2.6103, -2.1612, -1.6527, -2.1612, -1.6527, -0.8975, -1.9350, -0.8975,
        -1.9350, -0.8975, -1.9350, -0.8975, -1.9350, -0.8975, -1.9350, -0.8975,
        -1.6527, -2.1612, -1.6527, -1.6477, -2.1612, -1.6527, -0.8975, -1.9350,
        -0.8975, -0.9842, -0.9792, -0.9792, -0.9792, -0.9792, -0.9792, -0.9792,
        -1.6208, -0.9842, -0.9792, -0.9792, -0.5910, -0.5860, -0.5860, -0.5860,
        -0.5860, -0.5860, -0.5860, -0.9052, -0.8305, -0.8255, -0.9052, -0.8305,
        -0.8255, -0.8255, -0.8255, -0.8255, -0.8255, -0.8255, -0.8255, -0.8255,
        -0.8255, -0.8255, -0.8255, -0.8255, -0.8255, -0.8255, -0.8255, -0.8255,
        -0.8255, -0.8255, -0.8255, -0.8255, -0.8255, -0.8255, -0.8255, -0.8255,
        -0.8255, -0.8255, -0.8255, -0.8255, -0.9052, -0.5910, -0.5860, -0.5860,
        -0.5860, -0.5860, -0.5860, -0.5860, -0.5860, -0.9842, -0.9792, -0.9792,
        -0.9792, -0.9792, -0.9792, -0.9792, -0.9792, -0.5910, -0.5860, -1.2739,
        -0.5910, -0.5860, -0.5860, -0.58

In [45]:
# traj = states[0]
# traj = traj.unsqueeze(0).cuda()
traj = states[100:200].cuda()
traj_rep = traj_encoder(traj).squeeze().cpu().detach().numpy()
traj_rep_mean = np.mean(traj_rep, axis=0)

# (y1 + y2 + y3 + ... + yn) \dot phi(s) = 
# r_bar = \sum_i phi(s) \dot y_i

#(h_1, h_2), (h_3, h_4)
# run experiment

In [205]:
traj_rep

array([[-1.24519020e-02,  1.72140688e-01, -1.99941397e-01],
       [-7.23904744e-02, -1.54744750e-02, -5.95420599e-02],
       [ 6.85634539e-02,  2.61282235e-01,  1.13325939e-01],
       [ 4.77422550e-02,  2.72015452e-01,  1.10210940e-01],
       [ 1.82430655e-01, -1.45750400e-03, -1.27552032e-01],
       [ 1.63957328e-01, -1.02644628e-02, -1.24340072e-01],
       [ 1.34490222e-01,  7.30282515e-02,  1.17064044e-01],
       [ 1.76501513e-01,  6.12094477e-02,  1.24106809e-01],
       [ 1.07569657e-01,  7.68294409e-02, -2.65226930e-01],
       [-1.78889744e-02,  6.24545068e-02, -2.82744646e-01],
       [ 1.63143605e-01,  1.12280563e-01,  6.55186623e-02],
       [ 1.22181095e-01,  1.06202856e-01,  1.35962829e-01],
       [ 8.61239657e-02,  2.09408313e-01,  1.51084587e-01],
       [ 3.67255211e-02,  2.22995579e-01,  1.49353132e-01],
       [ 2.80162320e-02,  1.06925145e-01, -5.48024625e-02],
       [ 1.56395346e-01, -6.31937459e-02,  7.06259459e-02],
       [ 6.45730272e-02,  9.48007870e-03

In [46]:
traj_rep_mean

array([ 0.06979584,  0.10707922, -0.07742313], dtype=float32)

In [41]:
rewards[0]

tensor([-8.1057e-01,  7.1262e+00, -1.2540e+00, -3.4946e-01,  9.0002e+00,
         2.1126e-01, -1.8535e+00,  8.8510e+00,  8.9352e+00,  8.6656e+00,
        -2.5827e+00, -2.6289e-01,  4.2332e-01,  8.7441e+00, -2.0102e-02,
        -1.5102e-02,  4.1297e-01,  8.2360e+00, -3.0451e-02, -6.3241e-01,
        -2.0166e-01,  8.5588e+00, -1.3791e-01,  8.8788e+00,  9.0242e+00,
         7.3036e-01,  9.4242e+00,  9.0852e+00,  1.7579e-01,  8.5158e+00,
        -8.1698e-01, -3.5267e-01, -2.6763e-01, -6.5813e-01, -3.0128e-01,
         6.8619e-02,  4.3842e-01,  2.9023e-01,  8.5500e+00, -1.1962e+00,
        -3.1842e-01,  9.1133e-02, -2.1953e+00,  7.1932e-01,  9.4436e+00,
        -2.0102e-02, -2.1347e+00,  8.2675e+00, -9.5441e-01, -6.4952e-01,
         7.4832e+00, -1.7861e+01,  2.5501e-01,  8.9929e+00, -1.8842e-01,
         7.3283e+00, -6.3184e-01, -9.7307e-02, -2.7508e-01, -8.6688e-01,
        -3.7937e-01, -5.0416e-01, -4.9916e-01, -3.1089e+00, -5.9699e-01,
        -3.1842e-01, -3.8226e-01, -3.1842e-01, -5.6

In [44]:
pred_rewards = net(states[:100].cuda())
pred_rewards

tensor([[ 0.8634,  1.4445,  1.0318,  ...,  0.5792,  0.6320,  0.5792],
        [-1.2571,  0.6464,  1.3215,  ...,  0.4213,  0.5534,  0.5534],
        [ 0.5265,  0.9712,  1.0901,  ...,  3.6919,  0.3974,  1.4401],
        ...,
        [ 4.1567,  0.5214,  3.7496,  ..., -7.1280,  1.6360,  1.5472],
        [-0.1773,  1.7541,  1.8369,  ..., -0.2573,  1.5700,  3.6288],
        [ 2.3829,  1.9300,  3.0253,  ...,  1.1985,  1.2554, -4.0485]],
       device='cuda:1', grad_fn=<ViewBackward0>)

In [47]:
import importlib
importlib.reload(sys.modules['new_block_env_new'])
import new_block_env_new

In [48]:
pred_env = new_block_env_new.NewBlockEnv(traj_rep_mean, state_encoder)

In [49]:
ground_truth_env = new_block_env_new.NewBlockEnv(ground_truths[100])

In [50]:
pred_model = stable_baselines3.PPO("MlpPolicy", pred_env, verbose=1)
pred_model.learn(total_timesteps=100000)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 151      |
|    ep_rew_mean     | 22       |
| time/              |          |
|    fps             | 340      |
|    iterations      | 1        |
|    time_elapsed    | 6        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 151         |
|    ep_rew_mean          | 21.6        |
| time/                   |             |
|    fps                  | 287         |
|    iterations           | 2           |
|    time_elapsed         | 14          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.007909408 |
|    clip_fraction        | 0.0515      |
|    clip_range           | 0.2         |
|    entropy_loss  

In [51]:
ground_truth_model = stable_baselines3.PPO("MlpPolicy", ground_truth_env, verbose=1)
ground_truth_model.learn(total_timesteps=100000)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 151      |
|    ep_rew_mean     | 72.6     |
| time/              |          |
|    fps             | 370      |
|    iterations      | 1        |
|    time_elapsed    | 5        |
|    total_timesteps | 2048     |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 151        |
|    ep_rew_mean          | 78.1       |
| time/                   |            |
|    fps                  | 309        |
|    iterations           | 2          |
|    time_elapsed         | 13         |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.00788867 |
|    clip_fraction        | 0.0579     |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.6

In [52]:
random_model = stable_baselines3.PPO("MlpPolicy", pred_env, verbose=1)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [53]:
mean_ground_truth_pred_model_ret = stable_baselines3.common.evaluation.evaluate_policy(pred_model, ground_truth_env)
mean_ground_truth_ret = stable_baselines3.common.evaluation.evaluate_policy(ground_truth_model, ground_truth_env)

In [54]:
random_ret = stable_baselines3.common.evaluation.evaluate_policy(random_model, ground_truth_env)

In [55]:
mean_ground_truth_pred_model_ret

(143.8041330468608, 72.21269711054589)

In [56]:
mean_ground_truth_ret

(549.2371598946862, 248.8745421746163)

In [57]:
random_ret

(115.1812754392624, 95.47248270320816)

In [22]:
rewards = torch.Tensor(np.load(f'rewards_{NUM_EXAMPLES}.npy'))

In [246]:
rewards[400:500]

tensor([[14.5259,  0.3506,  0.5823,  ..., -0.2862, 12.8873, -0.9817],
        [ 0.3247, 13.8860, -0.4223,  ...,  6.9286,  0.0697,  6.4828],
        [-1.2136, -0.3162,  0.1728,  ..., -0.2736, -0.3433, -0.5525],
        ...,
        [18.5774,  3.2003,  9.5013,  ...,  1.7962, 16.8114,  1.1337],
        [11.1175, -0.2806, -0.2412,  ...,  0.1771,  0.1821,  0.2178],
        [-5.0372, 11.2712,  0.1843,  ..., -0.4950, -0.4900, -0.5960]])

In [14]:
# states_unsqueezed = states.unsqueeze(0)

In [131]:
# reward_est = net(traj)
# reward_est

tensor([[-7.3212e-01, -4.2512e+00, -2.6973e-02,  1.0328e-01, -1.0953e+00,
          4.8710e-01,  5.4005e-01,  5.7493e-01,  1.2628e-01,  1.2628e-01,
          5.0202e-02, -7.1855e-01, -3.4224e-01, -8.1891e-01, -5.0904e-01,
         -4.7608e+00,  6.4353e-01,  7.0998e-01,  5.0325e-01,  4.1208e-02,
          3.0775e-02, -2.0921e+00,  2.9576e-01,  1.2114e+00,  1.0126e+00,
          4.2032e-01, -2.3389e-01,  7.6685e-01, -3.2055e-01, -5.2329e-01,
         -4.6502e-01,  2.4208e-01,  9.9387e-02,  1.6569e-01,  4.0163e-01,
          1.6569e-01,  4.0163e-01,  3.3810e-01,  2.9431e-01, -6.5101e-01,
         -2.8945e-02, -4.5167e-01,  1.0377e-01, -4.5860e-01, -4.6851e-01,
         -5.8422e-01,  1.8117e-01,  5.6911e-01, -1.4262e-02,  4.6621e-01,
          4.4458e-01, -2.2552e-01,  1.8201e-01,  2.5598e-01,  1.2653e-02,
          2.1578e-01,  1.4733e-02,  2.6513e-01,  6.9023e-01,  1.2212e+00,
         -4.1603e-01, -6.7864e-01, -7.0866e-01, -2.0903e-01,  1.8196e-01,
         -1.2323e+00,  3.3696e-01,  1.

In [135]:
# reward = rewards[10002]
# reward

tensor([ 1.6892e+01, -7.5993e+00,  1.6548e+01,  1.6233e+00,  1.6377e+01,
         5.6338e-01,  7.9045e-01,  1.5786e+01, -1.5634e-01, -1.5134e-01,
        -4.7780e-01, -1.0134e+00, -6.8890e-01, -8.4088e-01, -8.3999e-01,
        -9.5788e+00, -4.2671e-01, -3.9427e-01,  7.0796e-02,  2.9113e-01,
         1.5066e+01, -8.2290e+00,  3.7405e-03,  1.4460e+01, -1.7538e+00,
        -5.8169e-01,  5.4681e-01,  1.3823e+00,  1.7491e+01,  1.6961e+01,
         1.5728e+01, -5.4159e-01, -5.5270e-01, -7.9088e-01, -9.0397e-01,
        -7.9088e-01, -9.0397e-01, -7.7421e-01, -1.0931e+00, -3.1113e-01,
         5.1612e-01,  1.5750e+01,  6.2372e-01,  1.5319e+01, -7.1023e-01,
        -1.0680e+00, -5.1854e-01, -7.4197e-01, -1.1471e-01,  1.5070e+01,
        -1.4794e+00, -1.3766e+00, -1.6370e+00, -1.2793e+00, -7.0576e-01,
         4.1434e-01,  1.5984e+01,  1.6020e+01,  6.0438e-01,  8.9786e-01,
         1.7299e+01,  1.6360e+01, -1.3591e-01,  1.5239e-02,  2.2079e-01,
         1.5721e+01, -6.9267e-01, -2.8673e-01, -5.7

In [120]:
# state_rep = state_encoder(traj[0][0])
# state_rep

tensor([-8.8296,  1.5263, -9.6218], device='cuda:3', grad_fn=<AddBackward0>)

In [16]:
# traj[0][0].view(5, 5, 4)

tensor([[[1., 0., 0., 0.],
         [1., 0., 0., 0.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.]],

        [[0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.]],

        [[0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.]],

        [[1., 0., 0., 0.],
         [0., 0., 0., 1.],
         [1., 0., 0., 0.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.]],

        [[0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [0., 0., 0., 1.],
         [1., 0., 0., 0.],
         [0., 0., 0., 1.]]], device='cuda:1')

In [114]:
# torch.dot(state_rep, traj_rep.squeeze())

tensor(-0.4094, device='cuda:3', grad_fn=<DotBackward0>)

In [132]:
# for i in range(100):
#     traj_rep = traj_encoder(traj)
#     print(torch.dot(state_rep, traj_rep.squeeze()))

tensor(-0.7321, device='cuda:3', grad_fn=<DotBackward0>)
tensor(-0.7321, device='cuda:3', grad_fn=<DotBackward0>)
tensor(-0.7321, device='cuda:3', grad_fn=<DotBackward0>)
tensor(-0.7321, device='cuda:3', grad_fn=<DotBackward0>)
tensor(-0.7321, device='cuda:3', grad_fn=<DotBackward0>)
tensor(-0.7321, device='cuda:3', grad_fn=<DotBackward0>)
tensor(-0.7321, device='cuda:3', grad_fn=<DotBackward0>)
tensor(-0.7321, device='cuda:3', grad_fn=<DotBackward0>)
tensor(-0.7321, device='cuda:3', grad_fn=<DotBackward0>)
tensor(-0.7321, device='cuda:3', grad_fn=<DotBackward0>)
tensor(-0.7321, device='cuda:3', grad_fn=<DotBackward0>)
tensor(-0.7321, device='cuda:3', grad_fn=<DotBackward0>)
tensor(-0.7321, device='cuda:3', grad_fn=<DotBackward0>)
tensor(-0.7321, device='cuda:3', grad_fn=<DotBackward0>)
tensor(-0.7321, device='cuda:3', grad_fn=<DotBackward0>)
tensor(-0.7321, device='cuda:3', grad_fn=<DotBackward0>)
tensor(-0.7321, device='cuda:3', grad_fn=<DotBackward0>)
tensor(-0.7321, device='cuda:3'

In [32]:
# reward_layer = net.reward_layer

AttributeError: 'NonLinearNet' object has no attribute 'reward_layer'

In [18]:
# states_1010000 = np.load("states_1010000.npy")

In [19]:
# states_10100 = states_1010000[:10100]

In [20]:
# np.save("states_10100", states_10100)

In [21]:
# rewards_1010000 = np.load("rewards_1010000.npy")

In [22]:
# rewards_10100 = rewards_1010000[:10100]

In [23]:
# np.save("rewards_10100", rewards_10100)